<a href="https://colab.research.google.com/github/bluecityisu/class2022/blob/main/eng_lstm_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install tensorflowjs

In [ ]:
!pip install beautifulsoup4

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
import string
import re
import csv
import numpy as np

In [ ]:
from bs4 import BeautifulSoup
table=str.maketrans('', '', string.punctuation)

In [ ]:
stopwords = ["I","by","i"]

In [ ]:
# this is for chinese only
"""
def getch(s):
  str33=[]
  res = re.compile(r"([\u4e00-\u9fa5])")
  ret = res.split(s)
  for ch in ret:
    if ch!='' and ch!=' ':
      str33.append(ch)
  str33=np.array(str33)
  return str33
"""

In [ ]:
sentences=[]
labels=[]
with open('/content/eng3_j.csv', encoding='UTF-8') as csvfile:
  reader = csv.reader(csvfile, delimiter=",")
  for row in reader:
    labels.append(row[0])
    sentence = row[1].lower()
    sentence = sentence.replace("_", " _ ")
    sentence = sentence.replace("/", " / ")
    soup = BeautifulSoup(sentence)
    sentence = soup.get_text()

    words = sentence.split()
    #words=getch(sentence)
    filtered_sentence = ""
    for word in words:
        word = word.translate(table)
        if word not in stopwords:
            filtered_sentence = filtered_sentence + word + " "
    sentences.append(filtered_sentence)

print(len(labels))
print(len(sentences))
print(sentences[5])

In [ ]:
sentences[9]

In [ ]:
vocab_size = 200
embedding_dim = 6
max_length = 50
trunc_type='post'
padding_type='post'
oov_tok = ""

In [ ]:
"""
sent2=[]
for nx in sentences:
  nn2=getch(nx)
  nn2=nn2.flatten()
  sent2.append(nn2)
"""

In [ ]:
#sent2[0]

In [ ]:
#sent2=np.array(sentences)
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)

tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index

In [ ]:
print(word_index)

In [ ]:
sentences

In [ ]:
sentences = tokenizer.texts_to_sequences(sentences)
print(sentences)
sentences=pad_sequences(sentences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(sentences)


In [ ]:
training_labels=[x.replace(" ", "") for x in labels]

In [ ]:
training_labels[0]=0

In [ ]:
for x in range(len(training_labels)):
  if training_labels[x]=='0':
    training_labels[x]=0
  elif training_labels[x]=='1':
    training_labels[x]=1
  elif training_labels[x]=='2':
    training_labels[x]=2
  else:
    training_labels[x]=int(training_labels[x])


In [ ]:
training_labels=np.array(training_labels)


In [ ]:
print(training_labels)

In [ ]:
ys = tf.keras.utils.to_categorical(training_labels, num_classes=5)

In [ ]:
print(len(ys))

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
    tf.keras.layers.Bidirectional(LSTM(max_length-1, return_sequences=True)),
    tf.keras.layers.Bidirectional(LSTM(max_length-1)),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
num_epochs = 400
history = model.fit(sentences, ys, epochs=num_epochs, verbose=2)

In [ ]:
cate=["Where are you shipping for?","$100","$30","$70","45"]
mylist=[]
nn3=[]
str3 = ["I want to use air freight","air freight taiwan","sea freight taiwan","sea freight"]
str3=np.array(str3)
for xx,yy in enumerate(str3):
  str3[xx]=yy.replace("?","")
  str3[xx]=yy.replace("I","")

str3 = tokenizer.texts_to_sequences(str3)
#str3=np.array(str3)
mylist = pad_sequences(str3, maxlen=max_length, padding=padding_type, truncating=trunc_type)


predict=model.predict(mylist)
print(np.round(predict,2))
print(np.argmax(predict,axis=1))
result=[cate[x] for x in np.argmax(predict,axis=1)]
print(result)


In [ ]:
def preds(x):
  mylist=[]
  str3=x.replace("?","")
  str3=np.array(str3)
  str3 = np.expand_dims(str3, axis=0)
  str3 = tokenizer.texts_to_sequences(str3)

  mylist = pad_sequences(str3, maxlen=max_length, padding=padding_type, truncating=trunc_type)
  predict=model.predict(mylist)

  pred_index=np.argmax(predict,axis=1)
  #print(pred_index)
  return cate[pred_index[0]]

In [ ]:
q1=input("air or sea?")
a1=preds(q1)

q2=input(a1)

a2=preds(q1 + " " + q2)
print(a2)

In [ ]:
model.save("/content/drive/MyDrive/class/tmp/rps.h5")

In [ ]:
#Load the model you save before
model = tf.keras.models.load_model('/content/drive/MyDrive/class/tmp/rps.h5')

In [ ]:
import tensorflowjs as tfjs

In [ ]:
tfjs.converters.save_keras_model(model, 'models')